In [1]:
# Load the CSV file
import os
import json
import io
import time
import contextlib
import pandas as pd
# from WIQACausalBuilder import WIQACausalBuilder
from DDXPlusCausalBuilder import DDXPlusCausalBuilder

from tqdm import tqdm

# ---- Log capture (JSON) ----
LOG_JSON_PATH = os.environ.get('WIQA_LOG_JSON_PATH', 'important_logs.json')
ALL_RUN_LOGS = []

def run_wiqa_with_log(datapoint, run_id=None, save_path=LOG_JSON_PATH, echo=True):
    buf = io.StringIO()
    wiqa = DDXPlusCausalBuilder(datapoint)
    started_at = time.time()
    pipeline_return = None
    error = None
    try:
        with contextlib.redirect_stdout(buf), contextlib.redirect_stderr(buf):
            pipeline_return = wiqa.run_wiqa_pipeline()
    except Exception as e:
        error = f'{type(e).__name__}: {e}'
    finished_at = time.time()
    log_text = buf.getvalue()

    record = {
        'run_id': run_id,
        'started_at': started_at,
        'finished_at': finished_at,
        'elapsed_s': finished_at - started_at,
        'datapoint': datapoint,
        'pipeline_return': pipeline_return,
        'is_correct': bool(pipeline_return) if pipeline_return is not None else False,
        'error': error,
        'cause_event': getattr(wiqa, 'cause_event', ''),
        'outcome_base': getattr(wiqa, 'outcome_base', ''),
        'log': log_text,
    }
    ALL_RUN_LOGS.append(record)

    if save_path:
        with open(save_path, 'w', encoding='utf-8') as f:
            json.dump(ALL_RUN_LOGS, f, ensure_ascii=False, indent=2, default=str)

    if echo and log_text:
        print(log_text, end='')
    return record

def save_all_logs(path=LOG_JSON_PATH):
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(ALL_RUN_LOGS, f, ensure_ascii=False, indent=2, default=str)
    print(f'Saved {len(ALL_RUN_LOGS)} runs to {path}')
# csv_path = r'E:\PHD\01\other_code\CDCR-SFT\data\wiqa_test.csv'
# df = pd.read_csv(csv_path)

In [4]:
datapoint = {'question_stem': 'If a patient experiences an Acute COPD exacerbation / infection and their oxygen saturation drops below 90%, how will this affect the probability of hospitalization?',
 'answer_label': 'more',
 'answer_label_as_choice': 'A',
 'choices': {'text': ['more', 'less', 'no_effect'], 'label': ['A', 'B', 'C']},}

log_record = run_wiqa_with_log(datapoint, run_id='pressure_cracks')
print(log_record['pipeline_return'])



步骤 0: 问题信息

问题: If a patient experiences an Acute COPD exacerbation / infection and their oxygen saturation drops below 90%, how will this affect the probability of hospitalization?
正确答案: more (A)

步骤 1: 提取起点和终点

提取结果:
  - 原因事件 (cause_event): 'a patient experiences an Acute COPD exacerbation / infection and their oxygen saturation drops below 90%'
  - 结果事件 (outcome_event): 'the probability of hospitalization'
  - 结果基底 (outcome_base): 'hospitalization probability'
  - 方向词 (outcome_direction_in_question): NONE
  - 是否否定 (outcome_is_negated): False

步骤 2: BFS 因果图扩展
is_same_variable_v2: hospitalization risk <-> hospitalization probability = close_hit (same_entity, same_quantity, direct_cause_or_effect)
is_same_variable_v2: oxygen therapy effectiveness <-> hospitalization probability = bridge_candidate (same_entity, different_quantity, direct_cause_or_effect)
is_same_variable_v2: patient stability <-> hospitalization probability = bridge_candidate (same_entity, different_quantity, direct_ca

In [ ]:
datapoint = {'question_stem': 'suppose during boiling point happens, how will it affect less evaporation.',
 'answer_label': 'less',
 'answer_label_as_choice': 'B',
 'choices': {'text': ['more', 'less', 'no_effect'], 'label': ['A', 'B', 'C']},}

log_record = run_wiqa_with_log(datapoint, run_id='boiling_evaporation')
print(log_record['pipeline_return'])
